In [11]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

mnist = input_data.read_data_sets("data/mnist", one_hot=True)


learning_rate = 0.01
batch_size = 128
n_epoches = 25

# input name space for tensorboard.
with tf.name_scope('input'):
    X = tf.placeholder(tf.float32, [batch_size, 784], name="image")
    Y = tf.placeholder(tf.float32, [batch_size, 10], name="label")

# train variables name space.
w = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.01), name="weight")
b = tf.Variable(tf.zeros([1, 10]), name="bias")

logits = tf.matmul(X, w) + b

# use softmax cross entropy with logits as the loss function
# compute the mean cross entropy, softmax is applied internally.
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
# computes the mean over examples in the batch
loss = tf.reduce_mean(entropy)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    sess.run(init)
# training the batch
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epoches):
        avg_cost = 0.
        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            _, l = sess.run([optimizer, loss], feed_dict={X: X_batch, Y:Y_batch})
            # Compute average loss
            avg_cost += l / n_batches
    print "Average cost {0}".format(avg_cost)
    
# testing the model
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, logits], feed_dict={X: X_batch, Y: Y_batch})
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
        total_correct_preds += sess.run(accuracy)
    print "Accuracy {0}".format(total_correct_preds/mnist.test.num_examples)
    

Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz
Average cost 0.345833116825
Accuracy 0.9103
